# DistilBERT Fine-Tuning for Comment Sentiment Classification

This notebook demonstrates how to fine-tune a DistilBERT model for sentiment classification of software engineering specific text using publiclly available datasets

# Cell 1: Setup Environment and Imports

In [1]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import display, HTML, clear_output

# Define project root path

In [3]:
import os
import sys
from pathlib import Path

# Try to detect the root directory of your project (where "src" folder exists)
def find_project_root(start: Path = Path.cwd()):
    for path in [start] + list(start.parents):
        if (path / "src").is_dir():
            return path
    raise RuntimeError("Project root with 'src' folder not found.")

# Set and add to sys.path
PROJECT_ROOT = find_project_root()
sys.path.append(str(PROJECT_ROOT))

# Now you can import config
from src.config_three_class import CONFIG_3CLASS

# Useful path shortcuts
DATA_DIR = PROJECT_ROOT / "data"
MODEL_DIR = PROJECT_ROOT / "saved_models"


# Cell 2: Import Model Classes and Functions

In [6]:
from src.models.distilbert.train_distilbert import main
from src.config_three_class import CONFIG_3CLASS as CONFIG

# Cell 3: Run Fine-tuning Process

In [7]:
# Run the main function and capture the returned fine_tuner instance
print("Starting fine-tuning process...")
fine_tuner = main()
print("Fine-tuning process completed!")   

Starting fine-tuning process...
[INFO] Initializing DistilBERT fine-tuner with model: distilbert-base-uncased
[INFO] Tokenizer loaded successfully.


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.

[INFO] Model distilbert-base-uncased initialized with 2 labels.
[INFO] Model moved to device: cpu
[INFO] Data module initialized.
[INFO] Model initialized on device: cpu
[INFO] Optimizer: AdamW with learning rate 1e-05
[INFO] ModelTrainer initialized.

[INFO] Starting DistilBERT fine-tuning process.
[INFO] Loading dataset...


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/preprocessed/combined_DeepSentimentSECrossPlatform.csv'

In [ ]:
# Cell 4: View Training Results

    
# Display metrics if available
if hasattr(fine_tuner, 'metrics'):
    print("Final Evaluation Metrics:")
    metrics_df = pd.DataFrame([fine_tuner.metrics])
    display(metrics_df.T.reset_index().rename(columns={'index': 'Metric', 0: 'Value'}))

# Visualize training history if available
if hasattr(fine_tuner.trainer, 'history') and fine_tuner.trainer.history:
    history = fine_tuner.trainer.history

    if history.get('train_loss') and history.get('val_loss'):
        # Create figure
        plt.figure(figsize=(14, 6))

        # Plot training and validation loss
        plt.subplot(1, 2, 1)
        plt.plot(history['train_loss'], 'b-o', label='Training Loss')
        plt.plot(history['val_loss'], 'r-o', label='Validation Loss')
        plt.title('Loss During Training')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True, alpha=0.3)

        # Plot performance metrics
        plt.subplot(1, 2, 2)
        if 'val_accuracy' in history:
            plt.plot(history['val_accuracy'], 'g-o', label='Validation Accuracy')
        if 'val_f1' in history:
            plt.plot(history['val_f1'], 'p-o', label='Validation F1')
        plt.title('Metrics During Training')
        plt.xlabel('Epoch')
        plt.ylabel('Score')
        plt.legend()
        plt.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()
else:
    print("No training history available.")
    


In [ ]:
# Cell 5: Run Comprehensive Evaluation
try:
    # Display header
    display(HTML("<div class='section-header'>Model Evaluation</div>"))
    
    # Import evaluator
    from src.training.model_evaluator import ModelEvaluator
    
    # Get test data
    print("Loading test dataset...")
    texts, labels = fine_tuner.data_module.load_dataset()
    _, _, test_loader = fine_tuner.data_module.create_dataloaders(texts, labels)
    print(f"Test dataset loaded with {len(test_loader.dataset)} samples.")
    
    # Create evaluator and run evaluation
    print("Running comprehensive evaluation...")
    evaluator = ModelEvaluator(
        model=fine_tuner.model,
        device=fine_tuner.device,
        model_name="DistilBERT Sentiment Classifier",
        class_names=fine_tuner.class_names
    )
    
    # Evaluate model (get raw results for custom visualizations)
    all_labels, all_preds, all_probs = evaluator.evaluate(test_loader)
    metrics = evaluator.calculate_metrics(all_labels, all_preds, all_probs)
    
    # Display confusion matrix
    plt.figure(figsize=(10, 8))
    cm = metrics["confusion_matrix"]
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
               xticklabels=fine_tuner.class_names, 
               yticklabels=fine_tuner.class_names)
    plt.title("Confusion Matrix")
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.tight_layout()
    plt.show()
    
    # Display classification metrics
    print("\nClassification Report:")
    metrics_df = pd.DataFrame({
        'Precision': [metrics['precision']],
        'Recall': [metrics['recall']],
        'F1 Score': [metrics['f1']],
        'Accuracy': [metrics['accuracy']]
    })
    display(metrics_df)
    
    # Display per-class metrics if available
    if 'per_class' in metrics:
        print("\nPer-Class Performance:")
        per_class_df = pd.DataFrame(metrics['per_class']).T
        display(per_class_df)
    
except Exception as e:
    print(f"An error occurred: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Cell 6: Generate PDF Report (Optional)
try:
    # Display header
    display(HTML("<div class='section-header'>Generate Evaluation Report</div>"))
    
    # Generate PDF report
    report_path = f"evaluation_results/{CONFIG['models']['distilbert']['pretrained_model_name']}_evaluation_report.pdf"
    os.makedirs(os.path.dirname(report_path), exist_ok=True)
    
    print(f"Generating comprehensive PDF report at {report_path}...")
    evaluator.generate_pdf_report(all_labels, all_preds, all_probs, output_path=report_path)
    print(f"PDF report generated successfully at: {report_path}")
    
except Exception as e:
    print(f"An error occurred: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
import torch

# Cell 7: Test with Custom Inputs (Optional)
try:
    # Display header
    display(HTML("<div class='section-header'>Test with Custom Inputs</div>"))
    
    def predict_sentiment(text):
        """Function to predict sentiment of a given text"""
        # Tokenize input
        inputs = fine_tuner.tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=CONFIG["training"]["max_length"]
        )
        
        # Move inputs to device
        inputs = {key: val.to(fine_tuner.device) for key, val in inputs.items()}
        
        # Set model to evaluation mode
        fine_tuner.model.eval()
        
        # Get prediction
        with torch.no_grad():
            outputs = fine_tuner.model(**inputs)
            logits = outputs.logits
            probabilities = torch.nn.functional.softmax(logits, dim=1)
            prediction = torch.argmax(logits, dim=1)
        
        # Get predicted class
        predicted_class = fine_tuner.class_names[prediction.item()]
        confidence = probabilities[0][prediction.item()].item() * 100
        
        # Get probabilities for all classes
        class_probs = {
            fine_tuner.class_names[i]: prob.item() * 100 
            for i, prob in enumerate(probabilities[0])
        }
        
        return predicted_class, confidence, class_probs
    
    # Create a simple UI for testing
    from ipywidgets import widgets, Layout
    from IPython.display import display, clear_output
    
    # Create text input
    text_input = widgets.Textarea(
        placeholder='Enter text to analyze...',
        layout=Layout(width='100%', height='100px')
    )
    
    # Create button
    button = widgets.Button(
        description='Analyze Sentiment',
        button_style='primary',
        layout=Layout(width='200px')
    )
    
    # Create output area
    output = widgets.Output()
    
    # Define button click event
    def on_button_click(b):
        with output:
            clear_output()
            if not text_input.value.strip():
                print("Please enter some text to analyze.")
                return
            
            print(f"Analyzing: \"{text_input.value}\"")
            predicted_class, confidence, class_probs = predict_sentiment(text_input.value)
            
            print(f"\nPredicted sentiment: {predicted_class} (Confidence: {confidence:.2f}%)")
            print("\nProbabilities for all classes:")
            for cls, prob in class_probs.items():
                print(f"  - {cls}: {prob:.2f}%")
    
    button.on_click(on_button_click)
    
    # Display widgets
    print("Try your own examples:")
    display(text_input)
    display(button)
    display(output)
    
except Exception as e:
    print(f"An error occurred: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
import torch
import os
import json


def save_model(fine_tuner, save_dir="saved_model"):
    """
    Save the model architecture, tokenizer, weights, and related configurations.
    """
    os.makedirs(save_dir, exist_ok=True)
    
    # Save model weights
    model_path = os.path.join(save_dir, "model.pth")
    torch.save(fine_tuner.model.state_dict(), model_path)
    print(f"Model weights saved at: {model_path}")
    
    # Save tokenizer
    tokenizer_path = os.path.join(save_dir, "tokenizer")
    fine_tuner.tokenizer.save_pretrained(tokenizer_path)
    print(f"Tokenizer saved at: {tokenizer_path}")
    
    # Save training history if available
    history_path = os.path.join(save_dir, "history.json")
    if hasattr(fine_tuner.trainer, 'history') and fine_tuner.trainer.history:
        with open(history_path, "w") as f:
            json.dump(fine_tuner.trainer.history, f, indent=4)
        print(f"Training history saved at: {history_path}")
    
    # Save model configuration
    config_path = os.path.join(save_dir, "config.json")
    config_data = {
        "model_name": "DistilBERT Sentiment Classifier",
        "class_names": fine_tuner.class_names,
        "device": fine_tuner.device,
        "pretrained_model": CONFIG["models"]["distilbert"]["pretrained_model_name"],
        "max_length": CONFIG["training"]["max_length"]
    }
    
    with open(config_path, "w") as f:
        json.dump(config_data, f, indent=4)
    print(f"Model configuration saved at: {config_path}")
    
    print("Model save process completed!")


In [ ]:
save_model(fine_tuner)
